## rsp 솔리디티 프로그램 작성후 배포
## 가위, 바위, 보를 각각 1,2,3로 설정 , play 에 가위 바위 보 승패 계산하는 알고리즘을 생각해 함수 구현
## 201810824 현소미

In [4]:
%%writefile src/Rsp1.sol
pragma solidity 0.4.25;
contract Rsp {
    address owner;
    mapping (address => uint) balanceOf;
    //address payable a_subject;
    address b_subject;
    uint8 a;
    uint8 b;
    constructor() public {
        owner = msg.sender;
    }
    function deposit(uint amount) payable public onlyOwner {
        balanceOf[msg.sender] += amount;
    }
     modifier onlyOwner {
        require(msg.sender == owner);
        _;
    }
    // forward from owner to another
    function forwardTo(address receiver, uint amount) payable public onlyOwner {
        require(balanceOf[msg.sender] >= amount);
        balanceOf[msg.sender] -= amount; // Subtract from the sender
        balanceOf[receiver] += amount;   // Add the same to the recipient
    }
     function getBalance() public view returns(uint, uint) {
        return (address(this).balance, balanceOf[owner]);
    }
    function getBalanceOf() public view returns (uint) {
        return balanceOf[b_subject];
    }
    function setA() public payable{
        a= uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%3)+1;
    }
    function setB(uint8 number) public payable returns(address addr){
        b=number;
        forwardTo(b_subject,1000);
        return b_subject;
    }
    function showA() public view returns(uint8){
        return a;
    }
    function showB() public view returns(uint8){
        return b;
    }
    function play() public view returns(uint8 ){
        uint8 result;
        uint8 re=(3+a-b)%3;
        if(re==0)   result=0;
        else if(re==1)  result=1;
        else    result=2;
        return result;
    }//1=a wins,2=b wins
    function distributeBetAmount(uint amount) payable public {
        if(play()==1){
        balanceOf[owner] += amount; 
        balanceOf[b_subject] -= amount; 
        }
        else if(play()==2){
        balanceOf[owner] -= amount; 
        balanceOf[b_subject] += amount; 
        }
        else {
             balanceOf[owner] ; 
            balanceOf[b_subject] ; 
        }
    }
    function getMatchResult() view public returns(string memory) {
        string memory result;
        uint8 re=(3+a-b)%3;
        if(re==0)   result="tie";
        else if(re==1)  result="A wins";
        else    result="B wins";
        return result;
    }
}

Overwriting src/Rsp1.sol


In [5]:
from IPython.display import Image

In [6]:
!solc --abi --bin --gas src/Rsp1.sol


======= src/Rsp1.sol:Rsp =======
Gas estimation:
construction:
   20898 + 600400 = 621298
external:
   deposit(uint256):	20941
   distributeBetAmount(uint256):	infinite
   forwardTo(address,uint256):	41642
   getBalance():	1167
   getBalanceOf():	948
   getMatchResult():	infinite
   play():	1069
   setA():	infinite
   setB(uint8):	62344
   showA():	610
   showB():	566
Binary: 
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610bba806100606000396000f3006080604052600436106100af576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806312065fe0146100b457806316abdf7d146100e65780631fc376f714610149578063437b0ce0146101535780634a68638114610184578063797890a41461021457806393e84cd914610245578063acfd68be14610276578063b6b55f2514610296578063b96ee716146102b6578063ecbde5e6146102f6575b600080fd5b3480156100c057600080fd5b506100c9610321565b60405180838152602001828

In [8]:
%%writefile src/Rsp1Deploy.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8446"));
var _abiArray =[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"number","type":"uint8"}],"name":"setB","outputs":[{"name":"addr","type":"address"}],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"setA","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"showB","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"getMatchResult","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"showA","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"play","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"distributeBetAmount","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"receiver","type":"address"},{"name":"amount","type":"uint256"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"getBalanceOf","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
//var _abiArray = JSON.parse(_abiStr);
var _bin="0x" + "608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610bba806100606000396000f3006080604052600436106100af576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806312065fe0146100b457806316abdf7d146100e65780631fc376f714610149578063437b0ce0146101535780634a68638114610184578063797890a41461021457806393e84cd914610245578063acfd68be14610276578063b6b55f2514610296578063b96ee716146102b6578063ecbde5e6146102f6575b600080fd5b3480156100c057600080fd5b506100c9610321565b604051808381526020018281526020019250505060405180910390f35b610107600480360381019080803560ff1690602001909291905050506103a5565b604051808273ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200191505060405180910390f35b61015161041a565b005b34801561015f57600080fd5b506101686104d3565b604051808260ff1660ff16815260200191505060405180910390f35b34801561019057600080fd5b506101996104ea565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156101d95780820151818401526020810190506101be565b50505050905090810190601f1680156102065780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b34801561022057600080fd5b506102296105f8565b604051808260ff1660ff16815260200191505060405180910390f35b34801561025157600080fd5b5061025a61060f565b604051808260ff1660ff16815260200191505060405180910390f35b61029460048036038101908080359060200190929190505050610681565b005b6102b460048036038101908080359060200190929190505050610933565b005b6102f4600480360381019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190803590602001909291905050506109de565b005b34801561030257600080fd5b5061030b610b25565b6040518082815260200191505060405180910390f35b6000803073ffffffffffffffffffffffffffffffffffffffff1631600160008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002054915091509091565b600081600260156101000a81548160ff021916908360ff1602179055506103f0600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff166103e86109de565b600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff169050919050565b60016003424460405160200180838152602001828152602001925050506040516020818303038152906040526040518082805190602001908083835b60208310151561047b5780518252602082019150602081019050602083039250610456565b6001836020036101000a0380198251168184511680821785525050505050509050019150506040518091039020600190048115156104b557fe5b0601600260146101000a81548160ff021916908360ff160217905550565b6000600260159054906101000a900460ff16905090565b60608060006003600260159054906101000a900460ff16600260149054906101000a900460ff166003010360ff1681151561052157fe5b06905060008160ff16141561056d576040805190810160405280600381526020017f746965000000000000000000000000000000000000000000000000000000000081525091506105f0565b60018160ff1614156105b6576040805190810160405280600681526020017f412077696e73000000000000000000000000000000000000000000000000000081525091506105ef565b6040805190810160405280600681526020017f422077696e73000000000000000000000000000000000000000000000000000081525091505b5b819250505090565b6000600260149054906101000a900460ff16905090565b60008060006003600260159054906101000a900460ff16600260149054906101000a900460ff166003010360ff1681151561064657fe5b06905060008160ff16141561065e5760009150610679565b60018160ff1614156106735760019150610678565b600291505b5b819250505090565b600161068b61060f565b60ff1614156107765780600160008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055508060016000600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200190815260200160002060008282540392505081905550610930565b600261078061060f565b60ff16141561086b5780600160008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825403925050819055508060016000600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254019250508190555061092f565b600160008060009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000205060016000600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020505b5b50565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561098e57600080fd5b80600160003373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254019250508190555050565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16141515610a3957600080fd5b80600160003373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000205410151515610a8757600080fd5b80600160003373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016000206000828254039250508190555080600160008473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020600082825401925050819055505050565b600060016000600260009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001908152602001600020549050905600a165627a7a72305820887733b8fdf2f56c2d9030e391450d67281d46beb1c63b03fadab416e10e6a390029";

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    new web3.eth.Contract(_abiArray).deploy({data: _bin}).estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas);
    });
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 897428}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance) {
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting src/Rsp1Deploy.js


In [2]:
!geth --exec "eth.getTransactionReceipt('0x2c9ed520dc318227bc0818f2a260b2c0ccb14a89e2f0578734db817f42706759')" attach http://localhost:8446

{
  blockHash: "0x9ed8626e654679912ed1513937150a39844fcf7f0bd88d35bc9ba81f63a3f849",
  blockNumber: 3500,
  contractAddress: "0xf0e8f343d57f9d5c478b365ffa9a98506ee9bf1e",
  cumulativeGasUsed: 856989,
  from: "0x54e81eae4e6a73a977ee3d56e75df4e02b059455",
  gasUsed: 856989,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  root: "0xe23083e8f584d0ef31a2a393dff99d8ac9324158888ed1904ae6041716abdc16",
  to: null,
  transactionHash: "0x2c9ed520dc318227bc0818f2a260b2c0ccb14a89e2f0578734db817f42706759",
  tra

In [9]:
!node src/Rsp1Deploy.js

Deploying the contract from 0x54e81Eae4E6A73a977ee3D56e75DF4e02B059455
>> gas: 871138
hash: 0xc59ba2633a8a5831e93a4507c92fe5e1bd9e0c6238191dacf97eaa6cda28e555
---> The contract deployed to: 0x7daC94CC7c7F806CC68903c9Fb678c664c678b6e


## use 단계

In [1]:
%%writefile src/Rsp3Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8446"));
var _abiArray=[{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"name":"","type":"uint256"},{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"number","type":"uint8"}],"name":"setB","outputs":[{"name":"addr","type":"address"}],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"setA","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"showB","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"getMatchResult","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"showA","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"play","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"distributeBetAmount","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"receiver","type":"address"},{"name":"amount","type":"uint256"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"getBalanceOf","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
web3.eth.personal.unlockAccount("0x54e81Eae4E6A73a977ee3D56e75DF4e02B059455","0902",1000).then(console.log('unlocked!'));
var Ex14 = new web3.eth.Contract(_abiArray,"0x7daC94CC7c7F806CC68903c9Fb678c664c678b6e");
async function doIt() {
    const accounts = await web3.eth.getAccounts();
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("contract balance: " + balanceBefore);
    console.log("contract balance, A balance ?: ");
    await Ex14.methods.deposit(2000).send({from:"0x54e81Eae4E6A73a977ee3D56e75DF4e02B059455",gas:871138});
    await Ex14.methods.getBalance().send({from:"0x54e81Eae4E6A73a977ee3D56e75DF4e02B059455",gas:871138});
    await Ex14.methods.getBalance().call().then(function(uint) {console.log(uint);});
    console.log("B balance ?:");
    await Ex14.methods.getBalanceOf().send({from:"0x54e81Eae4E6A73a977ee3D56e75DF4e02B059455",gas:871138});
    await Ex14.methods.getBalanceOf().call().then(function(uint) {console.log(uint);});
    await Ex14.methods.setA().send({from:"0x54e81Eae4E6A73a977ee3D56e75DF4e02B059455",gas:871138});
    await Ex14.methods.setB(2).send({from:"0x54e81Eae4E6A73a977ee3D56e75DF4e02B059455",gas:871138});
    console.log("B balance ?:");
    await Ex14.methods.getBalanceOf().send({from:"0x54e81Eae4E6A73a977ee3D56e75DF4e02B059455",gas:871138});
    await Ex14.methods.getBalanceOf().call().then(function(uint) {console.log(uint);});
    console.log("A rsp value (1=scissors, 2=rock, 3=paper)?:: ");
    await Ex14.methods.showA().send({from:"0x54e81Eae4E6A73a977ee3D56e75DF4e02B059455",gas:871138});
    await Ex14.methods.showA().call().then(function(uint) {console.log(uint);});
    console.log("B rsp value set to 2(=rock)!(1=scissors, 2=rock, 3=paper)?:: ");
    await Ex14.methods.showB().send({from:"0x54e81Eae4E6A73a977ee3D56e75DF4e02B059455",gas:871138});
    await Ex14.methods.showB().call().then(function(uint) {console.log(uint);});
    await Ex14.methods.play().send({from:"0x54e81Eae4E6A73a977ee3D56e75DF4e02B059455",gas:871138});
    console.log("play value(0=tie, 1=a wins, 2=b wins)?::");
    await Ex14.methods.play().call().then(function(uint) {console.log(uint);});
    await Ex14.methods.getMatchResult().send({from:"0x54e81Eae4E6A73a977ee3D56e75DF4e02B059455",gas:871138});
    await Ex14.methods.getMatchResult().call().then(function(str) {console.log(str);});
    await Ex14.methods.distributeBetAmount(1000).send({from:"0x54e81Eae4E6A73a977ee3D56e75DF4e02B059455",gas:871138});
    console.log("contract balance, A balance?:");
    await Ex14.methods.getBalance().send({from:"0x54e81Eae4E6A73a977ee3D56e75DF4e02B059455",gas:871138});
    await Ex14.methods.getBalance().call().then(function(uint) {console.log(uint);});
    console.log("B balance?:");
    await Ex14.methods.getBalanceOf().send({from:"0x54e81Eae4E6A73a977ee3D56e75DF4e02B059455",gas:871138});
    await Ex14.methods.getBalanceOf().call().then(function(uint) {console.log(uint);});
}

doIt()

Overwriting src/Rsp3Use.js


## 처음 a에 2000만큼 금액을 deposit해준후에 setb함수를 불러올때 b에게 1000만큼을 송금하도록 함수를 설계해서 b_balance가 0에서 1000으로 제대로 할당됨을 보였고 , a가 이기자 a의 잔고가 2000으로 늘어나고 b의 잔고는 0이 되어 제대로 금액 전달이 됐음을 확인했습니다. 

In [2]:
!node src/Rsp3Use.js

unlocked!
contract balance: 33090700000000000000000
contract balance, A balance ?: 
Result { '0': '0', '1': '2000' }
B balance ?:
2000
B balance ?:
3000
A rsp value (1=scissors, 2=rock, 3=paper)?:: 
1
B rsp value set to 2(=rock)!(1=scissors, 2=rock, 3=paper)?:: 
2
play value(0=tie, 1=a wins, 2=b wins)?::
2
B wins
contract balance, A balance?:
Result { '0': '0', '1': '0' }
B balance?:
4000
